In [5]:
import numpy as np
import pandas as pd
import torch
import logging
from tqdm import tqdm
import math
import argparse
import os

!git clone https://github.com/huggingface/transformers
!pip install transformers/
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

fatal: destination path 'transformers' already exists and is not an empty directory.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8049542 sha256=f447f5d87870eb5cc15b65820d43d6c559db82a4c23925e37ae235fe4d20c9df
  Stored in directory: /tmp/pip-ephem-wheel-cache-7ovlgnmd/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.0.dev0
    Uninstalling transformers-4.36.0.dev0:
      Successfully uninstalled transformers-4.36.0.dev0


In [22]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [23]:
parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=88888)
parser.add_argument("--model_name", default="gpt2", type=str)
parser.add_argument("--max_seq_length", default=512, type=int)
parser.add_argument("--train_batch_size", default=4, type=int)
parser.add_argument("--valid_batch_size", default=4, type=int)
parser.add_argument("--num_train_epochs", default=1, type=int)
parser.add_argument("--warmup", default=0.1, type=float)
parser.add_argument("--learning_rate", default=5e-5, type=float)
parser.add_argument("--input_text_path", default='drive/MyDrive/story-text', type=str)
args, _ = parser.parse_known_args()

In [24]:
!ls drive/MyDrive/story-text

test.wp_source	test.wp_target


In [1]:
DATAPATH=args.input_text_path
# def combinetext(prompt, story):
#     fp=open(os.path.join(DATAPATH,prompt),encoding='utf8')
#     fs=open(os.path.join(DATAPATH,story),encoding='utf8')
#     prompts=fp.readlines()
#     stories=fs.readlines()
#     assert len(prompts)==len(stories)
#     combine=[]
#     for i in range(len(prompts)):
#         combine.append(prompts[i].rstrip()+' <sep> '+" ".join(stories[i].split()[:300]))
#     return combine

#do a little text clean with punctuations
# def cleanpunctuation(s):
#     for p in '!,.:;?':
#         s=s.replace(' '+p,p)
#     s=s.replace(' '+'n\'t','n\'t')
#     s=s.replace(' '+'\'s','\'s')
#     s=s.replace(' '+'\'re','\'re')
#     s=s.replace(' '+'\'ve','\'ve')
#     s=s.replace(' '+'\'ll','\'ll')
#     s=s.replace(' '+'\'am','\'am')
#     s=s.replace(' '+'\'m','\'m')
#     s=s.replace(' '+'\' m','\'m')
#     s=s.replace(' '+'\'m','\'m')
#     s=s.replace(' '+'\' ve','\'ve')
#     s=s.replace(' '+'\' s','\'s')
#     s=s.replace('<newline>','\n')
#     return s
new version of code for parquet:

import re
import pandas as pd

def format_lines(prompts_file, completions_file, output_file):
    # Define a regular expression pattern to remove content in brackets
    brackets_pattern = re.compile(r"\[.*?\]\s*")

    # Initialize an empty list to store formatted data
    data = []

    with open(prompts_file, "r") as pf, open(completions_file, "r") as cf:
        for prompt, completion in zip(pf, cf):
            # Remove bracketed content from the prompt
            prompt = brackets_pattern.sub("", prompt.strip())
            completion = completion.strip()
            formatted_line = f"<s>[INST] {prompt} [/INST] {completion} </s>"
            data.append(formatted_line)

    # Convert list to DataFrame
    df = pd.DataFrame(data, columns=["formatted_text"])

    # Save DataFrame as Parquet file
    df.to_parquet(output_file, index=False)

# Specify the paths to your input and output files
prompts_file = "test.wp_source"
completions_file = "test.wp_target"
output_parquet_file = "train.parquet"

format_lines(prompts_file, completions_file, output_parquet_file)

# valid_text=combinetext('test.wp_source', 'test.wp_target')
# valid_text=list(map(cleanpunctuation,valid_text))

SyntaxError: ignored

In [37]:
with open('formatted_lines.txt', 'r') as file:
    for i in range(100):
        line = file.readline()
        print(line, end='')

<s>[INST] Leonardo DiCaprio in a fit of rage begins to torpedo his own career by deliberately acting poorly and taking on bad films . He finally wins an oscar for starring in Paul Blart : Mall Cop 3 . [/INST] The wet marble floor pressed on his cheek like a thousand hands slapping his face frozen in time . Smattering piss of rain ignored his indignant mumblings . His eyes fluttered . Pins and needs ran from finger to shoulder as he pushed back against the floor , contorting his aching body into a cross legged position . Last night was bad . He gathered that . His routine dullness of though crept inwards from the edges of his mind toward the black mist that veiled his most recent memories . He struggled to recall whatever he could n't recall but only for a moment before he decided it probably was n't worth the effort . <newline> He glanced around the room for a few minutes before concluding that he probably did n't know where he was . His investigation was n't entirely fruitless , he di

In [2]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nishankgite","key":"41766f49e0c1e5bdfe2573bd5fc15f6c"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d emily2008/story-text

100% 368M/370M [00:07<00:00, 23.6MB/s]
100% 370M/370M [00:07<00:00, 51.5MB/s]


In [5]:
!unzip story-text.zip

Archive:  story-text.zip
  inflating: test.wp_source          
  inflating: test.wp_target          
  inflating: train.wp_source         
  inflating: train.wp_target         
  inflating: valid.wp_source         
  inflating: valid.wp_target         


In [6]:
#new version of code for parquet:

import re
import pandas as pd

def format_lines(prompts_file, completions_file, output_file):
    # Define a regular expression pattern to remove content in brackets
    brackets_pattern = re.compile(r"\[.*?\]\s*")

    # Initialize an empty list to store formatted data
    data = []

    with open(prompts_file, "r") as pf, open(completions_file, "r") as cf:
        for prompt, completion in zip(pf, cf):
            # Remove bracketed content from the prompt
            prompt = brackets_pattern.sub("", prompt.strip())
            completion = completion.strip()
            formatted_line = f"<s>[INST] {prompt} [/INST] {completion} </s>"
            data.append(formatted_line)

    # Convert list to DataFrame
    df = pd.DataFrame(data, columns=["formatted_text"])

    # Save DataFrame as Parquet file
    df.to_parquet(output_file, index=False)

# Specify the paths to your input and output files
prompts_file = "train.wp_source"
completions_file = "train.wp_target"
output_parquet_file = "train.parquet"

format_lines(prompts_file, completions_file, output_parquet_file)

In [43]:
import re

def format_lines(prompts_file, completions_file, output_file):
    # Define a regular expression pattern to remove content in brackets
    brackets_pattern = re.compile(r"\[.*?\]\s*")

    with open(prompts_file, "r") as pf, open(completions_file, "r") as cf, open(output_file, "w") as of:
        for prompt, completion in zip(pf, cf):
            # Remove bracketed content from the prompt
            prompt = brackets_pattern.sub("", prompt.strip())
            completion = completion.strip()
            formatted_line = f"<s>[INST] {prompt} [/INST] {completion} </s>"
            of.write(formatted_line + "\n")

# Specify the paths to your input and output files
prompts_file = "test.wp_source"
completions_file = "test.wp_target"
output_file = "formatted_lines.txt"

format_lines(prompts_file, completions_file, output_file)

In [8]:
with open('train.parquet', 'r') as file:
    for i in range(100):
        line = file.readline()
        print(line, end='')

UnicodeDecodeError: ignored

In [9]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset('siddrao11/test')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['formatted_text'],
        num_rows: 272600
    })
    validation: Dataset({
        features: ['formatted_text'],
        num_rows: 15620
    })
    test: Dataset({
        features: ['formatted_text'],
        num_rows: 15138
    })
})


In [17]:
test_data = dataset['test']
print(test_data[26]['formatted_text'])

<s>[INST] You just started a new job in a large office building and what you do n't know is that at twelve o'clock every day , there is a office wide Purge style battle to the death , those who survive an hour get a significant pay raise and can come back tomorrow . A bell just went off , it 's twelve o'clock . [/INST] As I finished up my research on Alligator breeding habits for a story I was tasked with writing , a bell began to ring loudly throughout the office . I could feel the sound vibrating off the cubicle walls . I looked over my cubicle wall to ask a co-worker what the bell was for . <newline> <newline> I watched as he calmly opened his desk drawer , to reveal a small armory . There were multiple handguns , knives and magazines and other assorted weapons neatly stashed away . <newline> <newline> `` What the hell is that for ? '' I questioned loudly , and nervously . <newline> <newline> The man looked me in the eyes , and pointed his handgun at my face . I saw my life flash be

In [18]:
# Open a text file in write mode
with open('output.txt', 'w') as file:
    # Iterate through the first 1000 elements
    for i in range(1000):
        # Extract the 'formatted_text' from each element
        formatted_text = test_data[i]['formatted_text']
        # Write the text to the file, followed by a newline character
        file.write(formatted_text + '\n')
